# Game ot thrones
## Text Generation
<hr>

### Modelling

In [1]:
#import libraries
import os
import numpy as np
import pandas as pd
from pickle import dump

from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

2023-07-30 19:08:06.606341: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 19:08:15.931596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 19:08:28.642280: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Read the sequences

In [2]:
file = open('../datasets/got1_sequences.txt', 'r')
# read all text
txtDataset = file.read()
# close the file
file.close()
lstSequences = txtDataset.split('\n')

### Encode Sequences
The word embedding layer expects input sequences to be comprised of integers. We can map
each word in our vocabulary to a unique integer and encode our input sequences. Later, when
we make predictions, we can convert the prediction to numbers and look up their associated
words in the same mapping. To do this encoding, we will use the Tokenizer class in the Keras
API.

In [3]:
lstSequences

['a game of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint',
 'game of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of',
 'of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of a',
 'thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hi

In [3]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lstSequences)
sequences = tokenizer.texts_to_sequences(lstSequences)

In [13]:
sequences[0]

[4,
 1116,
 5,
 1741,
 1323,
 46,
 5,
 4,
 1032,
 5,
 602,
 2,
 248,
 65,
 3339,
 11938,
 11938,
 2403,
 11940,
 63,
 181,
 1367,
 57,
 1096,
 1207,
 16,
 1,
 791,
 252,
 3,
 1095,
 228,
 126,
 32,
 1,
 2138,
 55,
 148,
 59,
 1,
 148,
 2012,
 10,
 40,
 1322,
 889,
 143,
 18,
 230,
 1,
 2580]

In [4]:
vocab_size=len(tokenizer.word_index) + 1
print (f'Size of vocabulary : {vocab_size}')

Size of vocabulary : 11941


Now that we have encoded the input sequences, we need to separate them into input (X) and
output (y) elements, remember in the previos stage I split the text into a 50 words secuence + 1 to be the target label

In [5]:
dfsequences = pd.DataFrame(sequences, index=None)
dfsequences

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,4,1116,5,1741,1323,46,5,4,1032,5,...,40,1322,889,143,18,230,1,2580,NaN,NaN
1,1116,5,1741,1323,46,5,4,1032,5,602,...,1322,889,143,18,230,1,2580,5,NaN,NaN
2,5,1741,1323,46,5,4,1032,5,602,2,...,889,143,18,230,1,2580,5,4,NaN,NaN
3,1741,1323,46,5,4,1032,5,602,2,248,...,143,18,230,1,2580,5,4,274,NaN,NaN
4,1323,46,5,4,1032,5,602,2,248,65,...,18,230,1,2580,5,4,274,1096,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293481,3,9,190,9,102,3337,340,1005,11937,31,...,5,187,1,157,125,744,18,1,NaN,NaN
293482,9,190,9,102,3337,340,1005,11937,31,99,...,187,1,157,125,744,18,1,1742,NaN,NaN
293483,190,9,102,3337,340,1005,11937,31,99,285,...,1,157,125,744,18,1,1742,5,NaN,NaN
293484,9,102,3337,340,1005,11937,31,99,285,2,...,157,125,744,18,1,1742,5,536,NaN,NaN


In [6]:

X = dfsequences.iloc[:,0:49]
y = dfsequences.iloc[:,50]
y = to_categorical(y, num_classes=vocab_size)

seq_length = X.shape[1]

MemoryError: Unable to allocate 13.1 GiB for an array with shape (293486, 11941) and data type float32